# 👗 Clasificador y Generador de Reseñas de Prendas

Este sistema combina visión por computador y procesamiento de lenguaje natural para:

- 🔍 **Clasificar una prenda de ropa** en una de 15 categorías usando un modelo RexNet-150.
- ✍️ **Generar una reseña automática** usando un Transformer entrenado con descripciones reales.
- 🌐 **Interfaz Gradio** para subir imágenes y ver resultados en tiempo real.

### 🧠 Componentes:
- **Clasificador RexNet-150** (`modelo_final.pth`): predice clases como `Blazer`, `Jeans`, `Polo`, etc.
- **Generador Transformer** (`modelo_generativo_v2.pth`): crea una reseña basada en la clase predicha.
- **Vocabulario**: reconstruido desde `Reviews.csv`.

### ⚙️ Proceso:
1. Se sube una imagen.
2. Se clasifica la prenda con RexNet.
3. Se genera una reseña usando el Transformer.
4. Se muestra todo en una interfaz Gradio.

📍 Ideal para sistemas de recomendación de ropa y catálogos inteligentes.


In [ ]:
# === IMPORTS ===
import gradio as gr
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import timm
from PIL import Image
import pandas as pd
import math
from collections import Counter
import re
from nltk.tokenize import RegexpTokenizer
import os

# === SETUP ===
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
EMBED_SIZE = 512
NUM_HEADS = 4
NUM_LAYERS = 3
MAX_LEN = 128
DROPOUT = 0.1

# === CLASSES ===
CLOTHING_CLASSES = [
    'Blazer', 'Celana_Panjang', 'Celana_Pendek', 'Gaun', 'Hoodie',
    'Jaket', 'Jaket_Denim', 'Jaket_Olahraga', 'Jeans', 'Kaos',
    'Kemeja', 'Mantel', 'Polo', 'Rok', 'Sweter'
]

# === TOKENIZER & VOCAB ===
class DiccionarioTokens:
    def __init__(self, min_frecuencia=3):
        self.indice_a_token = {0: "[PAD]", 1: "[INICIO]", 2: "[FIN]", 3: "[DESCONOCIDO]", 4: ">>>"}
        self.token_a_indice = {v: k for k, v in self.indice_a_token.items()}
        self.min_frecuencia = min_frecuencia
        self.tokenizador = RegexpTokenizer(r'\w+|[^\w\s]+')

    def __len__(self):
        return len(self.indice_a_token)

    def construir_diccionario(self, lista_textos):
        contador = Counter()
        idx = len(self.indice_a_token)
        for texto in lista_textos:
            for token in self.procesar_texto(texto):
                contador[token] += 1
                if contador[token] >= self.min_frecuencia and token not in self.token_a_indice:
                    self.token_a_indice[token] = idx
                    self.indice_a_token[idx] = token
                    idx += 1

    def procesar_texto(self, texto):
        texto = texto.lower().strip()
        return [t for t in self.tokenizador.tokenize(texto) if t and not t.isspace()]

    def convertir_a_indices(self, texto):
        tokens = self.procesar_texto(texto)
        unk = self.token_a_indice["[DESCONOCIDO]"]
        return [self.token_a_indice.get(t, unk) for t in tokens]

# === POSITIONAL ENCODING ===
class CodificacionPosicional(nn.Module):
    def __init__(self, dim, max_len=MAX_LEN):
        super().__init__()
        pe = torch.zeros(max_len, dim)
        pos = torch.arange(0, max_len).unsqueeze(1).float()
        div = torch.exp(torch.arange(0, dim, 2).float() * (-math.log(10000.0) / dim))
        pe[:, 0::2] = torch.sin(pos / div)
        pe[:, 1::2] = torch.cos(pos / div)
        self.register_buffer('codificacion_pos', pe.unsqueeze(0))

    def forward(self, x):
        x = x + self.codificacion_pos[:, :x.size(1), :]
        return x

# === TRANSFORMER ===
class ModeloGeneradorTexto(nn.Module):
    def __init__(self, tamaño_vocabulario, dim_modelo, num_cabezas, num_capas):
        super().__init__()
        self.capa_embedding = nn.Embedding(tamaño_vocabulario, dim_modelo)
        self.escala_embedding = math.sqrt(dim_modelo)
        self.codificador_posicion = CodificacionPosicional(dim_modelo)
        capa_decoder = nn.TransformerDecoderLayer(
            d_model=dim_modelo, nhead=num_cabezas, dim_feedforward=dim_modelo*4,
            dropout=DROPOUT, activation='gelu', batch_first=True
        )
        self.decodificador = nn.TransformerDecoder(capa_decoder, num_layers=num_capas)
        self.normalizacion_final = nn.LayerNorm(dim_modelo)
        self.proyeccion_salida = nn.Linear(dim_modelo, tamaño_vocabulario)

    def crear_mascara_causal(self, tam):
        return torch.triu(torch.ones(tam, tam), diagonal=1).bool().to(DEVICE)

    def forward(self, tokens_entrada):
        emb = self.capa_embedding(tokens_entrada) * self.escala_embedding
        emb = self.codificador_posicion(emb)
        mascara = self.crear_mascara_causal(tokens_entrada.size(1))
        dec = self.decodificador(tgt=emb, memory=emb, tgt_mask=mascara)
        dec = self.normalizacion_final(dec)
        return self.proyeccion_salida(dec)

# === FUNCIONES AUXILIARES ===
def crear_vocab():
    df = pd.read_csv("/content/drive/MyDrive/ET deep learning/Reviews.csv")
    df = df.dropna(subset=['Review Text', 'Class Name'])
    textos = df['Class Name'].str.lower() + " >>> " + df['Review Text'].str.lower()
    vocab = DiccionarioTokens(min_frecuencia=3)
    vocab.construir_diccionario(textos.tolist())
    return vocab

def generar_reseña(modelo, clase, vocab, max_len=30):
    modelo.eval()
    tokens = [vocab.token_a_indice["[INICIO]"]] + vocab.convertir_a_indices(f"{clase.lower()} >>>")
    with torch.no_grad():
        for _ in range(max_len):
            entrada = torch.tensor(tokens, dtype=torch.long).unsqueeze(0).to(DEVICE)
            salida = modelo(entrada)
            next_token = torch.argmax(salida[0, -1]).item()
            tokens.append(next_token)
            if next_token == vocab.token_a_indice["[FIN]"]:
                break
    tokens = tokens[1:]  # quitar [INICIO]
    texto = " ".join(vocab.indice_a_token[t] for t in tokens if t not in [vocab.token_a_indice["[FIN]"], vocab.token_a_indice["[INICIO]"]])
    return texto.split(">>>")[-1].strip()

# === LOAD MODELS ===
def cargar_modelos():
    vocab = crear_vocab()

    # RexNet
    cnn_path = "/content/drive/MyDrive/ET deep learning/modelo_final2.pth"
    rexnet = timm.create_model("rexnet_150", pretrained=False, num_classes=len(CLOTHING_CLASSES))
    rexnet.load_state_dict(torch.load(cnn_path, map_location=DEVICE))
    rexnet = rexnet.to(DEVICE).eval()

    # Transformer
    transformer_path = "/content/drive/MyDrive/ET deep learning/modelo_generativo_v2.pth"
    checkpoint = torch.load(transformer_path, map_location=DEVICE)
    transformer = ModeloGeneradorTexto(len(vocab), EMBED_SIZE, NUM_HEADS, NUM_LAYERS)
    transformer.load_state_dict(checkpoint["model_state_dict"])
    transformer = transformer.to(DEVICE).eval()

    return rexnet, transformer, vocab

# === INFERENCIA ===
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def predecir(imagen):
    imagen = transform(imagen).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        out = rexnet(imagen)
        probs = torch.nn.functional.softmax(out, dim=1)
        idx = torch.argmax(probs, dim=1).item()
        clase = CLOTHING_CLASSES[idx]
        confianza = probs[0, idx].item()
        reseña = generar_reseña(transformer, clase, vocab)
    return clase, f"{confianza:.2%}", reseña

# === EJECUCIÓN ===
rexnet, transformer, vocab = cargar_modelos()

demo = gr.Interface(
    fn=predecir,
    inputs=gr.Image(type="pil"),
    outputs=[
        gr.Textbox(label="Clase de Prenda"),
        gr.Textbox(label="Confianza"),
        gr.Textbox(label="Reseña Generada")
    ],
    title="Clasificador de Prendas + Reseñas Transformer",
    description="Sube una imagen y obtén la clase de prenda + reseña generada"
)

demo.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://492f517066e25ef57a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Created dataset file at: .gradio/flagged/dataset1.csv


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2191, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1702, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^